In [338]:
import pandas as pd
import numpy as np
from dotenv import dotenv_values, find_dotenv
from functions import requirements_clean
import os
config = dotenv_values(find_dotenv())
path_rawdata = os.path.abspath(config["RAWDATA"]) + '\\'
path_cleandata = os.path.abspath(config["CLEANDATA"]) + '\\'

In [396]:
# 1. Load in Requirements Matrix 
requirements = pd.read_excel(path_rawdata + 'IxI_TR_2017_PRO_Det.xlsx', sheet_name='2017')
requirements = requirements_clean(requirements, wide=True)

In [435]:
# 2. Calculate 
requirements = requirements.fillna(value=0)
inverse = np.linalg.inv(requirements)
inverse = pd.DataFrame(inverse, index=requirements.index, columns=requirements.columns)
delta = np.identity(len(requirements)) - np.linalg.inv(requirements)
delta = pd.DataFrame(delta, index=requirements.index, columns=requirements.columns)
delta

desc_O,Abrasive product manufacturing,Accommodation,"Accounting, tax preparation, bookkeeping, and payroll services",Adhesive manufacturing,"Advertising, public relations, and related services",Air and gas compressor manufacturing,"Air conditioning, refrigeration, and warm air heating equipment manufacturing",Air transportation,Aircraft engine and engine parts manufacturing,Aircraft manufacturing,...,"Watch, clock, and other measuring and controlling device manufacturing",Water transportation,"Water, sewage and other systems",Wet corn milling,Wholesale electronic markets and agents and brokers,Wineries,Wired telecommunications carriers,Wireless telecommunications carriers (except satellite),Wiring device manufacturing,Wood kitchen cabinet and countertop manufacturing
desc_I,,,,,,,,,,,,,,,,,,,,,
Abrasive product manufacturing,4.838435e-02,9.318842e-07,-1.882491e-09,1.427448e-07,4.999278e-05,2.051756e-03,8.192560e-04,1.910384e-08,8.008479e-06,6.144023e-04,...,6.106306e-04,1.239428e-06,3.013055e-08,1.385162e-06,7.621191e-07,3.684732e-08,6.326541e-06,5.386979e-05,3.695521e-03,5.349353e-04
Accommodation,4.433206e-04,8.782860e-04,6.420952e-03,2.726814e-04,2.832052e-03,5.586366e-04,5.465710e-04,4.937779e-08,4.219587e-04,4.814026e-05,...,6.246258e-04,2.289266e-08,4.654048e-04,2.021727e-04,2.498898e-08,4.291665e-04,4.390385e-04,1.974276e-03,2.006911e-04,2.026386e-03
"Accounting, tax preparation, bookkeeping, and payroll services",-1.564845e-03,6.919956e-05,2.341234e-02,-2.485979e-03,-1.088884e-03,-2.129038e-03,-2.086506e-03,-2.029865e-03,-2.361163e-03,-2.844404e-03,...,-2.595623e-03,-1.120075e-03,1.375178e-02,-2.655621e-03,3.966931e-03,-7.139952e-04,4.686599e-04,6.843854e-03,-2.519363e-03,1.505510e-03
Adhesive manufacturing,8.255617e-03,1.667449e-05,-3.382686e-07,1.424915e-02,2.546605e-05,9.939708e-06,3.685872e-06,-3.865139e-07,-2.100042e-07,9.234220e-06,...,2.791751e-05,-2.404761e-07,3.446479e-07,1.484643e-06,-2.796244e-07,-1.225154e-07,2.691436e-07,6.607494e-06,9.075648e-06,4.085006e-03
"Advertising, public relations, and related services",1.376079e-03,7.761481e-03,1.883512e-03,1.157377e-03,5.910505e-03,8.926193e-04,9.608056e-04,1.679333e-03,8.956513e-04,5.613201e-04,...,1.426301e-03,7.132842e-03,8.843292e-05,2.710672e-04,1.100926e-02,2.335251e-03,6.276153e-03,9.135844e-03,7.176427e-04,1.878063e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wineries,-1.051265e-08,1.866339e-03,-4.819021e-08,1.347285e-07,3.284635e-08,3.214995e-08,2.350229e-08,1.128124e-03,-1.528075e-08,1.783888e-08,...,3.491629e-08,3.731305e-04,1.546079e-08,1.057410e-07,1.098673e-06,6.278298e-02,-2.848255e-08,-2.672644e-08,-4.399464e-08,-2.382627e-08
Wired telecommunications carriers,4.507040e-03,1.604435e-03,9.263768e-04,-2.166185e-04,1.953376e-03,1.159681e-03,7.329501e-04,2.861756e-04,1.804143e-03,9.594919e-04,...,-2.843033e-04,3.037773e-04,4.162586e-03,-7.746890e-04,1.706186e-03,2.157097e-05,5.924578e-02,5.419060e-02,-5.953301e-04,7.210425e-04
Wireless telecommunications carriers (except satellite),-2.409343e-04,1.017932e-03,1.317994e-03,-6.514459e-04,1.742959e-03,-4.798665e-04,-5.375590e-04,4.339132e-04,2.855713e-04,-2.032010e-04,...,-3.790782e-04,-3.203638e-04,5.928308e-04,-7.528909e-04,2.741746e-03,1.637150e-04,3.788111e-03,3.117566e-02,-3.925166e-04,8.837893e-04


In [441]:
# 3. Merge concordance 
concordance = pd.read_pickle("B:/OneDrive - DAZ/Desktop/BEAdata - Tony/cleandata/concordance/concordance6_naics6_addproportions.pkl")
# indsutry_to_product = concordance.set_index('NAICS_desc')['product'].to_dict()
product_row = pd.Series(indsutry_to_product, index=delta.columns)
delta.loc['Product'] = product_row
last_row = delta.iloc[-1:]
delta = pd.concat([last_row, delta])
delta.columns = delta.iloc[0]
delta = delta.iloc[1:-1]
delta

197

In [438]:
# 4. Collapse columns
delta = delta.groupby(delta.columns, axis=1).sum()
delta.to_excel("new.xlsx")

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_1556\2662056264.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  delta = delta.groupby(delta.columns, axis=1).sum()


In [259]:
# 5. Make Matrix to Sales
make_matrix = pd.read_excel("B:/OneDrive - DAZ/Desktop/BEAdata - Tony/rawdata/Supply_2017_DET.xlsx", sheet_name="2017")
make_matrix = make_matrix.iloc[3:,1:]
make_matrix = make_matrix.drop(make_matrix.index[1])
make_matrix.columns = make_matrix.iloc[0]
make_matrix = make_matrix.drop(make_matrix.index[0])
make_matrix.reset_index(drop=True, inplace=True)
make_matrix = make_matrix[[make_matrix.columns[0], 'Total Commodity Output']]
make_matrix.rename(columns={make_matrix.columns[0]: 'Industries', 'Total Commodity Output': "Sales"}, inplace=True)
make_matrix = make_matrix.fillna(value=0)
sales = make_matrix.iloc[:-3]

C:\Users\TonyGui\AppData\Local\Temp\ipykernel_1556\2833606589.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  make_matrix = make_matrix.fillna(value=0)


In [260]:
# 6. Merge concordance with sales 
product_sales = pd.merge(concordance, sales, left_on='NAICS_desc', right_on='Industries')
product_sales = product_sales[["product", "Industries", 'Sales']]
# 7. Group sum of Sales for every Product
product_sales_sum = product_sales.groupby('product', as_index=False)['Sales'].sum()
product_sales_sum.rename(columns={'Sales': "Sales Sums"}, inplace=True)

In [262]:
# Potential Problem
# product_sales[product_sales["product"] == "Repair and rental of audio-visual, photographic, and information processing equipment"]
# product_total = product_sales["Sales"].sum()
# product_total
# sales_total = sales["Sales"].sum()
# sales_total

33772566

In [279]:
# 8. Sales Share 
sale_shares = pd.merge(product_sales_sum, product_sales, right_on="product", left_on="product")
sale_shares = sale_shares[["product", "Industries", "Sales", "Sales Sums"]]
sale_shares["Sale Share"] = sale_shares["Sales"] / sale_shares["Sales Sums"]
sale_shares_products = sale_shares[["Industries", "Sale Share"]]

In [303]:
# 9. Merge Sale shares and Delta
sale_shares_products
delta_industry_shares = pd.merge(delta, sale_shares_products, left_index=True, right_on="Industries")

desc_O,Accessories and parts,Air and gas compressor manufacturing,Aircraft engine and engine parts manufacturing,Alcohol in purchased meals,"All other forging, stamping, and sintering","All other miscellaneous professional, scientific, and technical services",All other professional medical services,All other retail,Alumina refining and primary aluminum production,Aluminum product manufacturing from purchased aluminum,...,Valve and fittings other than plumbing,"Veneer, plywood, and engineered wood product manufacturing",Veterinary and other services for pets,Video and audio streaming and rental,"Video discs, tapes, and permanent digital downloads",Watches,Wholesale electronic markets and agents and brokers,Wine,Wiring device manufacturing,Wood kitchen cabinet and countertop manufacturing
desc_I,,,,,,,,,,,,,,,,,,,,,
"Tools, hardware, and supplies",2.720645e-03,2.051756e-03,8.008479e-06,0.001164,2.424281e-04,1.173701e-05,0.000038,6.286325e-06,2.354973e-05,2.120538e-04,...,1.403406e-03,6.392134e-04,1.199498e-06,2.765833e-05,2.945808e-07,6.106306e-04,7.621191e-07,3.684732e-08,3.695521e-03,5.349353e-04
Laundry and drycleaning services,1.859974e-03,5.586366e-04,4.219587e-04,0.013588,5.481836e-04,2.966938e-03,0.000876,3.219607e-05,2.545056e-04,3.778854e-04,...,5.510008e-04,5.838196e-04,3.032883e-03,1.384045e-02,1.711857e-05,6.246258e-04,2.498898e-08,4.291665e-04,2.006911e-04,2.026386e-03
Tax preparation and other related services,-1.089747e-02,-2.129038e-03,-2.361163e-03,0.026231,-1.992721e-03,4.808882e-03,0.001045,1.279036e-03,-2.510869e-03,-2.079973e-03,...,-1.768247e-03,-1.425959e-03,3.065634e-03,1.056736e-03,-2.575776e-03,-2.595623e-03,3.966931e-03,-7.139952e-04,-2.519363e-03,1.505510e-03
Stationery and miscellaneous printed materials,4.257506e-03,9.939708e-06,-2.100042e-07,0.000850,1.243801e-05,1.486279e-07,0.000077,9.935466e-06,2.525258e-04,1.084222e-06,...,4.750828e-06,3.351146e-02,5.293582e-04,1.377895e-04,2.766512e-07,2.791751e-05,-2.796244e-07,-1.225154e-07,9.075648e-06,4.085006e-03
Other personal business services,2.387082e-03,8.926193e-04,8.956513e-04,0.026021,6.672665e-04,4.686537e-03,0.005571,1.048203e-02,3.698466e-03,6.339799e-04,...,1.097420e-03,7.528181e-04,3.076482e-03,1.120318e-02,3.788881e-04,1.426301e-03,1.100926e-02,2.335251e-03,7.176427e-04,1.878063e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wine,3.178560e-08,3.214995e-08,-1.528075e-08,0.020081,-4.188014e-08,-2.816349e-08,0.001749,-2.896581e-08,2.031189e-08,6.924996e-09,...,-3.387085e-08,-4.762539e-08,-5.212186e-08,-1.796630e-08,8.695266e-10,3.491629e-08,1.098673e-06,6.278298e-02,-4.399464e-08,-2.382627e-08
Internet access,-2.705413e-03,1.159681e-03,1.804143e-03,0.008820,2.672860e-04,4.094355e-03,0.001764,1.618513e-03,-2.952557e-04,-5.404830e-04,...,8.988345e-04,4.514419e-04,3.072508e-03,4.018829e-03,-8.269020e-04,-2.843033e-04,1.706186e-03,2.157097e-05,-5.953301e-04,7.210425e-04
Cellular telephone services,-2.340546e-03,-4.798665e-04,2.855713e-04,0.008482,-4.990459e-04,3.645826e-03,0.000592,1.691169e-03,-8.494671e-04,-6.023856e-04,...,-3.356183e-05,-3.847433e-04,1.752348e-03,5.715907e-03,-8.703784e-04,-3.790782e-04,2.741746e-03,1.637150e-04,-3.925166e-04,8.837893e-04


In [305]:
delta.index

Index(['Tools, hardware, and supplies', 'Laundry and drycleaning services',
       'Tax preparation and other related services',
       'Stationery and miscellaneous printed materials',
       'Other personal business services',
       'Air and gas compressor manufacturing', 'Major household appliances',
       'Passenger fares for foreign travel',
       'Aircraft engine and engine parts manufacturing', 'Pleasure aircraft',
       ...
       'Watches', 'Passenger fares for foreign travel', 'Natural gas',
       'Sugar and sweets',
       'Wholesale electronic markets and agents and brokers', 'Wine',
       'Internet access', 'Cellular telephone services',
       'Wiring device manufacturing',
       'Wood kitchen cabinet and countertop manufacturing'],
      dtype='object', name='desc_I', length=402)